In [145]:
## Fixed issue: we should consider only first innings
## Pending issue: 
#  -what is the best way to fix division by 0 problems
#  -tau has too many zeros!!


### prep

In [1]:
import pickle
import pandas as pd
import numpy as np
from scipy.stats import dirichlet, multinomial

In [2]:
with open('df1.pkl', 'rb') as f:
    data = pickle.load(f)
df = pd.DataFrame(data)
df.head(10)

,bowler,batsman,wicket,over,outcome
0,Mortaza,Warner,0,1,0
1,Mortaza,Warner,0,1,0
2,Mortaza,Warner,0,1,1
3,Mortaza,Watson,0,1,0
4,Mortaza,Watson,0,1,1
5,Mortaza,Warner,0,1,0
6,Shafiul,Watson,0,2,0
7,Shafiul,Watson,0,2,1
8,Shafiul,Watson,0,2,0
9,Shafiul,Watson,0,2,2


In [3]:
X = np.load("X_mat.npy")

In [10]:
n_mat = X.sum(axis=3, keepdims=True)

In [11]:
n_mat.shape

(361, 20, 10, 1)

In [12]:
emp_prob = X / (n_mat+1.e-15)
# emp_prob.shape
np.max(emp_prob)

0.9999999999999999

In [13]:
## Delta theorem multipliers
inv_prob = np.array([(1-p)/(p+1.e-15) for p in emp_prob])
inv_prob = inv_prob/(n_mat+1.e-15)
# inv_prob.shape

### alpha matrix

In [14]:
alpha=[]
v_inv=[]
for i in range(emp_prob.shape[0]):
    alpha_i=[]
    v_inv_i=[]
    for o in range(19):
        val = emp_prob[i,o+1]/(emp_prob[i,o]+1.e-15)
        alpha_i.append(val)
        v_i = np.square(val)*(inv_prob[i,o+1]+inv_prob[i,o]) 
        v_inv_i.append([1/(np.sqrt(x)+1.e-15) for x in v_i])
        # v_i_root = np.sqrt(v_i)
        # print(v_i_root.shape)
        # v_inv_i.append(v_i_root)
    alpha.append(alpha_i)
    v_inv.append(v_inv_i)
    
alpha = np.array(alpha)
v_inv = np.array(v_inv)
v_inv.shape

(361, 19, 10, 8)

In [15]:
den = np.sum(v_inv, axis=0)
num = np.sum(v_inv*alpha, axis=0)
o_trans = num/den

In [16]:
np.min(o_trans)

0.0

### beta matrix

In [17]:
beta=[]
w_inv=[]
for i in range(emp_prob.shape[0]):
    beta_o=[]
    y_inv_o=[]
    for o in range(20):
        beta_i=[]
        v_inv_i=[]
        for w in range(9):
            val = emp_prob[i,o,w+1]/(emp_prob[i,o,w]+1.e-15)
            beta_i.append(val)
            v_i = np.square(val)*(inv_prob[i,o,w+1]+inv_prob[i,o,w]) 
            v_inv_i.append([1/(np.sqrt(x)+1.e-15) for x in v_i])
            # print(val.shape)
        beta_o.append(beta_i)
        y_inv_o.append(v_inv_i)
    beta.append(beta_o)
    w_inv.append(y_inv_o)
    
beta = np.array(beta)
w_inv = np.array(w_inv)
w_inv.shape 

(361, 20, 9, 8)

In [18]:
w_trans = np.sum(w_inv*beta, axis=0)/np.sum(w_inv, axis=0)
w_trans.shape

(20, 9, 8)

In [19]:
np.min(w_trans)

0.0

### tau calc

In [21]:
multiplier_mat = np.ones((2,20,10,8))
for o in range(1,20):
    multiplier_mat[0,o] = multiplier_mat[0,o-1]*o_trans[o-1]
for w in range(1,10):
    multiplier_mat[1,:,w] = multiplier_mat[1,:,w-1]*w_trans[:,w-1]

tau_mat = multiplier_mat[0]*multiplier_mat[1]
print(tau_mat[6,0])
for j in range(8):
    if tau_mat[6,0,j]!=0:
        tau_mat[:,:,j] = tau_mat[:,:,j]/(tau_mat[6,0,j]) # tau_mat[6,0,:]=1
    else:
        tau_mat[:,:,j] = (tau_mat[:,:,j])/(tau_mat[6,0,j]+1.e-15) 

[4.61995256e-089 1.92328359e-060 3.66193835e-100 0.00000000e+000
 6.93975012e-097 0.00000000e+000 1.51525237e-102 1.38649632e-101]


In [30]:
# tau_mat[18]

### parameter estimation

In [44]:
# Function to calculate the log posterior
def log_posterior(p, X, a, tau, m):
    log_p = np.sum([(np.sum(X[:, :, j]) + a[j] - 1) * np.log(p[j]) for j in range(len(p))])
    log_p -= np.sum([m[o, w] * np.log(np.sum([tau[o, w, j] * p[j] for j in range(len(p))])+1.e-15) for o in range(X.shape[0]) for w in range(X.shape[1])])
    return log_p

# Metropolis within Gibbs sampler
def metropolis_within_gibbs(X, a, tau, m, num_samples, burn_in):
    num_params = len(a)
    samples = np.zeros((num_samples, num_params))
    current_sample = np.ones(num_params) / num_params  # Initial sample
    for i in range(num_samples + burn_in):
        for j in range(num_params):
            proposal = np.random.dirichlet([np.sum(X[:, :, j]) + a[j] for _ in range(num_params)])
            acceptance_ratio = np.exp(log_posterior(proposal, X, a, tau, m) - log_posterior(current_sample, X, a, tau, m))
            if np.random.rand() < acceptance_ratio:
                current_sample = proposal
        if i >= burn_in:
            samples[i - burn_in] = current_sample
    return samples

In [51]:
def met_sampling(i):
    c = 60
    a = c * np.sum(X/(tau_mat+1.e-15), axis=tuple([0,1,2])) 
    a /= np.sum(X/(tau_mat+1.e-15))
    # tau = np.random.rand(3, 4, 8) 
    m = np.sum(X[i], axis=-1)
    num_samples = 1000
    burn_in = 100
    samples = metropolis_within_gibbs(X[i], a, tau_mat, m, num_samples,burn_in)
    return np.mean(samples, axis=0)

In [56]:
## Probability matrix calc
prob = []
for i in range(X.shape[0]):
    param_est = met_sampling(i)
    print(param_est)
    vals = tau_mat*param_est
    prob.append(vals/(vals.sum(axis=-1, keepdims=True)+1.e-15))

[1.98139444e-01 3.88146159e-01 5.50123213e-03 3.33186157e-01
 3.09429091e-02 4.39651407e-02 4.44833949e-05 7.44754969e-05]
[3.68188235e-01 5.87352499e-01 6.42932752e-05 4.13237789e-02
 2.28114757e-03 7.86340845e-04 1.72505031e-06 1.98110102e-06]
[0.22531271 0.36737483 0.07953325 0.05737121 0.05773193 0.1032595
 0.02906603 0.08035055]
[0.23911816 0.23322086 0.10576551 0.06615135 0.10081088 0.06825925
 0.09293816 0.09373583]
[0.18842648 0.29219102 0.14273933 0.04978439 0.04886525 0.0963155
 0.10204909 0.07962893]
[0.23307371 0.30793371 0.08713465 0.0508523  0.07685479 0.05539096
 0.10179818 0.0869617 ]
[0.35088896 0.20542535 0.07078165 0.09523815 0.15728564 0.05183087
 0.0052152  0.06333419]
[0.19829785 0.46000378 0.10613049 0.01491126 0.07720866 0.00919133
 0.04997494 0.08428168]
[0.28407327 0.23215784 0.06730352 0.1491913  0.12475055 0.09623453
 0.01573651 0.03055247]
[0.45262862 0.35149578 0.05476666 0.00792998 0.07222643 0.04714887
 0.01160002 0.00220365]
[0.14371427 0.46622385 0.054

KeyboardInterrupt: 

In [ ]:
np.save("prob_mat.npy", prob)

In [ ]:
# Modifications: - weigh recent matches more (use decay ratio)